<a href="https://colab.research.google.com/github/mohammed22habib18-lgtm/Medical_Prescription/blob/main/Medical_Prescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import hashlib, json, time, os
from dataclasses import dataclass, asdict
from typing import List, Optional

DATA_FILE = "prescriptions_chain.json"

def sha256(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

@dataclass
class Prescription:
    prescription_id: str
    patient_name: str
    medication: str
    dosage: str
    issued_on: str
    doctor: str
    status: str  # ISSUED or REVOKED
    remarks: str = ""

    def serialize(self) -> str:
        payload = {
            "prescription_id": self.prescription_id,
            "patient_name": self.patient_name,
            "medication": self.medication,
            "dosage": self.dosage,
            "issued_on": self.issued_on,
            "doctor": self.doctor,
            "status": self.status,
            "remarks": self.remarks,
        }
        return json.dumps(payload, separators=(",", ":"), sort_keys=True)

@dataclass
class Block:
    index: int
    timestamp: float
    data: dict
    prev_hash: str
    nonce: int = 0

    def hash(self) -> str:
        body = {
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "prev_hash": self.prev_hash,
            "nonce": self.nonce,
        }
        return sha256(json.dumps(body, separators=(",", ":"), sort_keys=True))

class Blockchain:
    def __init__(self):
        self.chain: List[Block] = []
        self.load() or self._create_genesis()

    def _create_genesis(self):
        genesis = Block(
            index=0,
            timestamp=time.time(),
            data={"type": "GENESIS", "msg": "Medical Prescription Ledger Genesis"},
            prev_hash="0"*64,
        )
        self.chain = [genesis]
        self.save()

    def last_block(self) -> Block:
        return self.chain[-1]

    def add_block(self, data: dict) -> Block:
        new_block = Block(
            index=len(self.chain),
            timestamp=time.time(),
            data=data,
            prev_hash=self.last_block().hash()
        )
        while not new_block.hash().startswith("00"):
            new_block.nonce += 1
        self.chain.append(new_block)
        self.save()
        return new_block

    def is_valid(self) -> bool:
        if not self.chain:
            return False
        for i in range(1, len(self.chain)):
            cur = self.chain[i]
            prev = self.chain[i-1]
            if cur.prev_hash != prev.hash():
                return False
            if not cur.hash().startswith("00"):
                return False
        return True

    def save(self):
        with open(DATA_FILE, "w") as f:
            out = [asdict(b) for b in self.chain]
            json.dump(out, f, indent=2)

    def load(self) -> bool:
        if not os.path.exists(DATA_FILE):
            return False
        with open(DATA_FILE, "r") as f:
            raw = json.load(f)
            self.chain = [Block(**b) for b in raw]
        return True

    def find_latest_prescription(self, prescription_id: str) -> Optional[Prescription]:
        for b in reversed(self.chain):
            if b.data.get("type") in ("ISSUE", "REVOKE"):
                p = b.data["prescription"]
                if p["prescription_id"] == prescription_id:
                    return Prescription(**p)
        return None

    def issue_prescription(self, p: Prescription) -> str:
        existing = self.find_latest_prescription(p.prescription_id)
        if existing and existing.status == "ISSUED":
            return f"[X] Prescription {p.prescription_id} already ISSUED."
        data = {"type": "ISSUE", "prescription": json.loads(p.serialize())}
        blk = self.add_block(data)
        return f"[✓] Issued {p.prescription_id} in block #{blk.index}."

    def revoke_prescription(self, prescription_id: str, doctor: str, remarks: str) -> str:
        current = self.find_latest_prescription(prescription_id)
        if not current:
            return f"[X] Prescription {prescription_id} not found."
        if current.status == "REVOKED":
            return f"[X] Prescription {prescription_id} already REVOKED."
        revoked = Prescription(
            prescription_id=prescription_id,
            patient_name=current.patient_name,
            medication=current.medication,
            dosage=current.dosage,
            issued_on=current.issued_on,
            doctor=doctor,
            status="REVOKED",
            remarks=remarks
        )
        data = {"type": "REVOKE", "prescription": json.loads(revoked.serialize())}
        blk = self.add_block(data)
        return f"[✓] Revoked {prescription_id} in block #{blk.index}."

    def verify_prescription(self, prescription_id: str) -> str:
        p = self.find_latest_prescription(prescription_id)
        if not p:
            return f"[?] {prescription_id} not found in ledger."
        for b in reversed(self.chain):
            if b.data.get("type") in ("ISSUE", "REVOKE"):
                pp = b.data["prescription"]
                if pp["prescription_id"] == prescription_id:
                    ok_link = b.prev_hash == self.chain[b.index-1].hash() if b.index > 0 else True
                    ok_pow = b.hash().startswith("00")
                    state = f"VALID LINK={ok_link}, POW={ok_pow}"
                    return f"[INFO] {prescription_id}: status={pp['status']} (doctor={pp['doctor']}). {state}."
        return f"[?] {prescription_id} not found in ledger."

def menu():
    bc = Blockchain()
    while True:
        print("\n--- Medical Prescription Ledger ---")
        print("1) Issue prescription")
        print("2) Revoke prescription")
        print("3) Verify prescription")
        print("4) Show chain length & validity")
        print("5) List last N blocks")
        print("0) Exit")
        choice = input("Select: ").strip()

        if choice == "1":
            pid = input("Prescription ID: ").strip()
            patient = input("Patient name: ").strip()
            med = input("Medication: ").strip()
            dosage = input("Dosage: ").strip()
            date = input("Issued on (YYYY-MM-DD): ").strip()
            doctor = input("Doctor: ").strip()
            msg = bc.issue_prescription(Prescription(pid, patient, med, dosage, date, doctor, "ISSUED"))
            print(msg)

        elif choice == "2":
            pid = input("Prescription ID to revoke: ").strip()
            doctor = input("Doctor confirming revocation: ").strip()
            remarks = input("Reason/remarks: ").strip()
            print(bc.revoke_prescription(pid, doctor, remarks))

        elif choice == "3":
            pid = input("Prescription ID to verify: ").strip()
            print(bc.verify_prescription(pid))

        elif choice == "4":
            print(f"Blocks: {len(bc.chain)} | Valid: {bc.is_valid()}")

        elif choice == "5":
            try:
                n = int(input("How many recent blocks? ").strip())
            except ValueError:
                n = 5
            for b in bc.chain[-n:]:
                print(f"#{b.index} ts={int(b.timestamp)} prev[:8]={b.prev_hash[:8]} nonce={b.nonce}")
                print(f"   type={b.data.get('type')}")
                if b.data.get("type") in ("ISSUE", "REVOKE"):
                    p = b.data["prescription"]
                    print(f"   id={p['prescription_id']} status={p['status']} patient={p['patient_name']}")

        elif choice == "0":
            break
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    menu()



--- Medical Prescription Ledger ---
1) Issue prescription
2) Revoke prescription
3) Verify prescription
4) Show chain length & validity
5) List last N blocks
0) Exit
Select: 1
Prescription ID: RX001
Patient name: Alice Smith
Medication:  Amoxicillin
Dosage: 500mg twice daily
Issued on (YYYY-MM-DD): 2025-08-22
Doctor: Dr. John Miller
[✓] Issued RX001 in block #3.

--- Medical Prescription Ledger ---
1) Issue prescription
2) Revoke prescription
3) Verify prescription
4) Show chain length & validity
5) List last N blocks
0) Exit
Select: 3
Prescription ID to verify: RX001
[INFO] RX001: status=ISSUED (doctor=Dr. John Miller). VALID LINK=True, POW=True.

--- Medical Prescription Ledger ---
1) Issue prescription
2) Revoke prescription
3) Verify prescription
4) Show chain length & validity
5) List last N blocks
0) Exit
Select: 2
Prescription ID to revoke:  RX001
Doctor confirming revocation: Dr. John Miller
Reason/remarks: Allergy reported
[✓] Revoked RX001 in block #4.

--- Medical Prescript